In [1]:
import numpy as np
import pandas as pd
import glob
import time
from scipy import signal
from tqdm.notebook import tqdm

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve

In [5]:
channels = ['Fz', 'FCz', 'Cz', 'CPz', 'Pz', 'POz']

train_filtered = np.load('data/X_train_bwbs.npy')

f = 'data/TrainLabels.csv'
train_labels = pd.read_csv(f)
train_labels = train_labels.Prediction.values

In [6]:
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
#Using SVM to find the optimal C in the dataset with baseline
C_list = [0.001,0.01,0.1,10]

opt_C = None
scaler = StandardScaler()
best_Cs = []

start = time.time()
for i in range(len(train_filtered[0])):
    x_train, x_test, y_train, y_test = train_test_split(train_filtered[:,i,:], train_labels, test_size=0.2)
    
    auc=0
    for c in C_list:
        estimator = SVC(kernel = 'linear',max_iter=10000,probability=True)
        parameter = {'kernel':('linear',), 'C':[c]}
        grid_search = GridSearchCV(estimator, parameter, cv=5)
        grid_search.fit(scaler.fit_transform(x_train), y_train)
        pred_y = grid_search.predict(scaler.transform(x_test))
        auc_score = roc_auc_score(y_test, pred_y)
#         print('AUC:'+"{0:.3f}".format(auc_score))
        if auc_score > auc:
            auc = auc_score
            opt_C = c
    best_Cs.append(opt_C)

print('Time taken: {}'.format(time.time() - start))
for i in range(len(channels)):
    print('Best parameter C* for {} = {}'.format(channels[i], best_Cs[i]))